In [2]:
import pandas as pd
import sqlite3

In [1]:
!apt-get install -y ssh
!pip install colab_ssh --upgrade

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libwrap0 ncurses-term openssh-server openssh-sftp-server ssh-import-id
Suggested packages:
  molly-guard monkeysphere ssh-askpass ufw
The following NEW packages will be installed:
  libwrap0 ncurses-term openssh-server openssh-sftp-server ssh ssh-import-id
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 804 kB of archives.
After this operation, 6,291 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 openssh-sftp-server amd64 1:8.9p1-3ubuntu0.10 [38.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libwrap0 amd64 7.6.q-31build2 [47.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 openssh-server amd64 1:8.9p1-3ubuntu0.10 [435 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 ssh all 1:8.9p1-3ubuntu0.10 [4,

In [2]:
# 개인 키 파일 경로 설정
private_key_path = '/content/drive/MyDrive/ML4_adv/demo_96.pem'
!chmod 400 {private_key_path}

In [ ]:
# SSH 접속
!ssh -i /content/drive/MyDrive/ML4_adv/demo_96.pem -p 32170 root@10.196.197.36

In [3]:
# Load CSV files
within_500m_bus = pd.read_csv('/content/drive/MyDrive/ML4_adv/within_500m_bus.csv')
station_data = pd.read_csv('/content/drive/MyDrive/ML4_adv/역세권조사(haversine).csv')

<ipython-input-3-be2a84e8b179>:2: DtypeWarning: Columns (17,18,37) have mixed types. Specify dtype option on import or set low_memory=False.
  within_500m_bus = pd.read_csv('/content/drive/MyDrive/ML4_adv/within_500m_bus.csv')
<ipython-input-3-be2a84e8b179>:3: DtypeWarning: Columns (17,18,37) have mixed types. Specify dtype option on import or set low_memory=False.
  station_data = pd.read_csv('/content/drive/MyDrive/ML4_adv/역세권조사(haversine).csv')


In [ ]:
# 사용할 열만 선택
station_data = station_data[['시군구', '번지', 'within_250m', 'within_500m', 'beyond_500m']]

# 공통 열을 기준으로 병합하기
merged_df = pd.merge(within_500m_bus, station_data, how='inner', on=['시군구', '번지'])



In [ ]:
# SQLite 데이터베이스 연결 (새 데이터베이스 파일 생성)
conn = sqlite3.connect('/content/drive/MyDrive/temp_database.db')
cursor = conn.cursor()

# CSV 파일을 읽어서 SQLite 테이블로 가져오기
within_500m_bus = pd.read_csv('/content/drive/MyDrive/ML4_adv/within_500m_bus.csv')
station_data = pd.read_csv('/content/drive/MyDrive/ML4_adv/역세권조사(haversine).csv')

within_500m_bus.to_sql('within_500m_bus', conn, if_exists='replace', index=False)
station_data.to_sql('station_data', conn, if_exists='replace', index=False)

# SQL 쿼리를 사용하여 테이블 병합
query = '''
SELECT *
FROM within_500m_bus AS a
INNER JOIN station_data AS b
ON a.시군구 = b.시군구 AND a.번지 = b.번지
'''

merged_df = pd.read_sql_query(query, conn)



<ipython-input-5-6def54b2de19>:6: DtypeWarning: Columns (17,18,37) have mixed types. Specify dtype option on import or set low_memory=False.
  within_500m_bus = pd.read_csv('/content/drive/MyDrive/ML4_adv/within_500m_bus.csv')
<ipython-input-5-6def54b2de19>:7: DtypeWarning: Columns (17,18,37) have mixed types. Specify dtype option on import or set low_memory=False.
  station_data = pd.read_csv('/content/drive/MyDrive/ML4_adv/역세권조사(haversine).csv')


In [ ]:
# 병합된 데이터를 CSV 파일로 내보내기
merged_df.to_csv('/content/drive/MyDrive/merged_within_500m_bus_and_station.csv', index=False)

# 병합된 데이터프레임을 사용자에게 표시하기
import ace_tools as tools; tools.display_dataframe_to_user(name="Merged DataFrame", dataframe=merged_df)

# 데이터베이스 연결 종료
conn.close()

In [ ]:
# 청크 단위로 병합하기 위한 함수
def merge_chunks(bus_file, station_file, chunk_size, output_file):
    station_data = pd.read_csv(station_file)
    chunk_iter = pd.read_csv(bus_file, chunksize=chunk_size)

    with open(output_file, 'w') as f:
        first_chunk = True
        for chunk in chunk_iter:
            merged_chunk = pd.merge(chunk, station_data, how='inner', on=['시군구', '번지'])
            if first_chunk:
                merged_chunk.to_csv(f, index=False)
                first_chunk = False
            else:
                merged_chunk.to_csv(f, index=False, header=False, mode='a')

# 파일 경로 및 청크 크기 설정
bus_file = '/content/drive/MyDrive/ML4_adv/within_500m_bus.csv'
station_file = '/content/drive/MyDrive/ML4_adv/역세권조사(haversine).csv'
output_file = '/content/drive/MyDrive/ML4_adv/merged_within_500m_bus_and_station.csv'
chunk_size = 10000  # 청크 크기 조정 가능

# 청크 단위로 병합 실행
merge_chunks(bus_file, station_file, chunk_size, output_file)

# 병합된 최종 파일 읽어서 사용자에게 표시하기
merged_df = pd.read_csv(output_file)

<ipython-input-2-5da431d46d62>:3: DtypeWarning: Columns (17,18,37) have mixed types. Specify dtype option on import or set low_memory=False.
  station_data = pd.read_csv(station_file)


In [ ]:
merged_df